# 磁化率的测定

In [1]:
from os import path
import pandas as pd
import numpy as np

In [2]:
data_dir = path.join('..', '数据', '磁化率')
raw_data = pd.read_csv(path.join(data_dir, '磁化率.csv'))
raw_data

,样品,m1H0,m1H1,m1H2,m2H2,m2H1,m2H0
0,空样品管,19.3233,19.3224,19.3221,19.3223,19.3226,19.3237
1,NaN,19.3236,19.3226,19.3224,19.3224,19.3225,19.3235
2,莫尔盐+空管,25.4385,25.5072,25.5315,25.5315,25.5074,25.4386
3,NaN,25.4388,25.5072,25.5319,25.5319,25.5072,25.4388
4,FeSO4•7H2O+空管,25.2196,25.3174,25.3516,25.3517,25.3174,25.2197
5,NaN,25.2198,25.3174,25.3519,25.3520,25.3170,25.2199
6,K3Fe(CN)6+空管,25.7340,25.7494,25.7551,25.7551,25.7495,25.7340
7,NaN,25.7340,25.7493,25.7550,25.7550,25.7495,25.7341
8,K4Fe(CN)6•3H2O,25.5149,25.5134,25.5129,25.5128,25.5135,25.5149
9,NaN,25.5150,25.5136,25.5130,25.5128,25.5134,25.5149


In [3]:
# raw_data.loc[raw_data.index % 2 == 0] += raw_data.loc[raw_data.index % 2 == 1].to_numpy()
raw_data.loc[raw_data.index % 2 == 0]

# ['m1H0':'m2H0'] /= 2

v1 = (raw_data.loc[raw_data.index % 2 == 0])[raw_data.columns[1:]].to_numpy() # 第一次测量的数值
v2 = (raw_data.loc[raw_data.index % 2 == 1])[raw_data.columns[1:]].to_numpy() # 第二次测量的数值
avg = pd.DataFrame((v1 + v2) / 2, columns=raw_data.columns[1:], index=raw_data['样品'][raw_data.index % 2 == 0])
avg # 取得均值

,m1H0,m1H1,m1H2,m2H2,m2H1,m2H0
样品,,,,,,
空样品管,19.32345,19.32250,19.32225,19.32235,19.32255,19.32360
莫尔盐+空管,25.43865,25.50720,25.53170,25.53170,25.50730,25.43870
FeSO4•7H2O+空管,25.21970,25.31740,25.35175,25.35185,25.31720,25.21980
K3Fe(CN)6+空管,25.73400,25.74935,25.75505,25.75505,25.74950,25.73405
K4Fe(CN)6•3H2O,25.51495,25.51350,25.51295,25.51280,25.51345,25.51490


In [4]:
avg['dm1H1'] = (avg['m1H1'] - avg['m1H0'] + avg['m2H1'] - avg['m2H0']) / 2
avg['dm1H2'] = (avg['m1H2'] - avg['m1H0'] + avg['m2H2'] - avg['m2H0']) / 2
avg[avg.columns[6:]]

,dm1H1,dm1H2
样品,,
空样品管,-0.001000,-0.001225
莫尔盐+空管,0.068575,0.093025
FeSO4•7H2O+空管,0.097550,0.132050
K3Fe(CN)6+空管,0.015400,0.021025
K4Fe(CN)6•3H2O,-0.001450,-0.002050


## 计算磁化率和人相应的励磁电流下的磁场强度值

$$
\chi_M = \frac{9500}{T + 1} \times 4\pi \times 10^{-12} \times M
$$

$$
H = \sqrt{\frac{2\left( \Delta m_{1} - \Delta m_{2} \right)ghM}{\chi_M\mu_0m}}
$$

In [8]:
# 定义实验环境信息
T = 25 + 273.15 # 室温，K
g = 9.8 # 重力加速度，m/s^2
h = 10.5 / 100 # 样品高度
M = 392.13 * 10 ** -3 # 摩尔盐M
avg['M'] = np.array([np.nan, 392.13, 278.05, 329.24, 422.388])
mu_0 = 4 * np.pi * 10**(-7) # 磁导率，T*m/A

In [11]:
avg['chi_M'] = 9500 / (T + 1) * 4 * np.pi * 10 ** (-12) * avg['M']
avg['m'] = (avg['m1H0'] + avg['m2H0'] - avg['m1H0']['空样品管'] - avg['m2H0']['空样品管']) / 2
avg['H1'] = np.sqrt(2 * (np.abs(avg['dm1H1'] - avg['dm1H1']['空样品管'])) * g * h * avg['M'] / (mu_0 * avg['chi_M']  * avg['m']))
avg['H2'] = np.sqrt(2 * (np.abs(avg['dm1H2'] - avg['dm1H2']['空样品管'])) * g * h * avg['M'] / (mu_0 * avg['chi_M']  * avg['m']))
avg.iloc[1:4][avg.columns[8:]]

,M,chi_M,m,H1,H2
样品,,,,,
莫尔盐+空管,392.13,1.564857e-07,6.115150,6.833113e+06,7.953030e+06
FeSO4•7H2O+空管,278.05,1.109602e-07,5.896225,8.282030e+06,9.631254e+06
K3Fe(CN)6+空管,329.24,1.313884e-07,6.410500,3.240198e+06,3.774110e+06


## 计算$\mu_m$和未成对电子数$n$

$$
\chi_M = \frac{N_0\mu_m^2\mu_0}{3kT}
$$

$$
\mu_m = \sqrt{\frac{3kT\chi_M}{N_0\mu_0}}
$$

$$
\mu_m = \sqrt{n(n + 2)}\mu_B
$$